In [ ]:
%load_ext autoreload
%autoreload 2


import logging


logging.basicConfig(
    format='%(asctime)s %(name)s %(levelname)-8s %(message)s',
    level=logging.DEBUG,
    datefmt='%Y-%m-%d %H:%M:%S')

logger = logging.getLogger("fab_duckdb")
logger.setLevel(logging.DEBUG)



In [ ]:
import duckdb

import fabduckdb
fabduckdb.register()

createstatement = "CREATE or REPLACE TABLE abc as SELECT * FROM range(1,10,1) t(x);CREATE or REPLACE TABLE def as SELECT * FROM range(20,30,2) t(x)"
loopstatement = "loop (select * from {{x}}) over (select x from (values('abc'), ('def')) t(x))"
duckdb.default_connection.execute(createstatement)
duckdb.default_connection.execute(loopstatement).df()

In [ ]:
# Dynamic SQL Example

import duckdb

import fabduckdb
fabduckdb.register()

statement = "execute (select 'select ' || i || ' as r, x from range(' || i || ') t(x)' from range(4) t(i)) using ('union all by name')"

duckdb.default_connection.execute(statement).df()

In [ ]:
import duckdb
from collections import deque


def functionreplace(query):
    """
    Transforms this statement: 
        select * from somefunction(arguments)
    Into:
        Python:
            df123 = somefunction(arguments);
        SQL:
            select * from df123
    """

def ifstatement(query):
    """
    Transforms this statement: 
        if (statement returning a single boolean) do (statement1) [else (statement2)]
    Into:
        result = (statement returning a single boolean)
        if result:
            return statement1
        else:
            return statement2
    """
    


def rewrite_statement(query: str):
    pass

def newexecute(query: str, parameters: object = None, multiple_parameter_sets: bool = False, connection: duckdb.DuckDBPyConnection = None) -> duckdb.DuckDBPyConnection:  # type: ignore
    # I am your captain now

    # Remove comments
    statementQueue = deque()

    # Split Statements
    statements = []
    map(statementQueue.append, statements)

    while statementQueue:
        statement = statementQueue.popleft()
        print(f"Processing: {statement}")

        new_statements = rewrite(statement)

        if new_statements is None: # run as is
            duckdb._orig_execute(statement)
        else:
            map(statementQueue.appendleft, new_statements)

def add_statement(keyword: str, method: function):
    ADDED_STATEMENTS[keyword] = method

if not hasattr(duckdb, "_orig_execute"):
    duckdb._orig_execute = duckdb.execute

duckdb.execute=newexecute

duckdb.add_statement = add_statement

duckdb.ADDED_STATEMENTS = {}
